<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [84]:
import pandas as pd
import psycopg2
import plotly
import plotly.express as px
import requests
from sqlalchemy import create_engine
from bs4 import BeautifulSoup


In [141]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

NameError: name 'DBNAME' is not defined

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [87]:
# текст запроса
query_3_1 = f'''select
                    count(id)
                from vacancies
'''

In [88]:
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [89]:
query_3_2 = f'''select
                    count(id)
                from employers
'''

In [90]:
count_empl = pd.read_sql_query(query_3_2, connection)
count_empl

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [91]:
query_3_3 = f'''select
                    count(id)
                from areas
'''

In [92]:
count_area = pd.read_sql_query(query_3_3, connection)
count_area

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [93]:
query_3_4 = f'''select
                    count(id)
                from industries
'''

In [94]:
count_ind = pd.read_sql_query(query_3_4,connection)
count_ind

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,294


***

*Из разведывательного анализа можно сделать вывод о том что 23501 компания представляет 49197 вакансий, следовательно не заглядывая мы можем предположить что каждая из компаний выставила по 2 IT вакансии на сайт, возможно компания представлена в нескольких регионах страны, либо требуются разные специалисты. Также представлено множество различных сфер деятельности, но исходя из общего количества вакансий, проглядывается то, что данные вакансии смежны между собой*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [95]:
n = 5
query_4_1 = f'''select
                    count(v.id) cnt,
                a.name area
                from vacancies v
                    join areas a on a.id = v.area_id
                group by a.name
                order by 1 desc
                limit {n}
'''

In [96]:
most_vacan = pd.read_sql_query(query_4_1, connection)
display(most_vacan)


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [97]:
query_4_2 = f'''select
                    count(id)
                from vacancies 
                where salary_from IS NOT NULL or salary_to IS NOT NULL
'''

In [98]:
fill_salary = pd.read_sql_query(query_4_2, connection)
print(fill_salary)

   count
0  24073


c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [99]:
query_4_3 = f'''select
                    round(avg(salary_from)),
                    round(avg(salary_to))
                from vacancies
'''

In [100]:
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_salary

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [101]:
query_4_4 = f'''select
                    count(id),
                    schedule,
                    employment
                from vacancies 
                group by schedule, employment
                order by 1 desc
'''

In [102]:
group_job_type = pd.read_sql_query(query_4_4, connection)
display(group_job_type)  # или же display(group_job_type.iloc[1])

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [103]:
query_4_5 = f'''select
                    experience "Требуемый опыт работы"
                from vacancies 
                group by experience
                order by count(id)
'''

In [104]:
exp_needed = pd.read_sql_query(query_4_5, connection)
exp_needed

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Требуемый опыт работы
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

*Из проделанного анализа можно сделать вывод о том, что большинство вакансий располагаются в городах миллиониках, где базируются основные IT компании. Помимо этого из анализа можно сделать вывод, что большинство работадателей заинтересованно в подготовленных кадрах, которые имеют опыт работы в данной сфере. Тенденция о работе на полный день еще сохраняется, но уже очень многие компании переводят IT-специалистов на удаленный график. Из анализа зарплат можно сделать вывод, что средняя зарплата будет составлять примерно 90 тысяч, по всем имеющимся вакансиям.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [105]:
query_5_1 = f'''(select      
                    e.name
                from employers e
                join vacancies v on v.employer_id = e.id
                group by e.id
                order by count(v.id) desc
                limit 1)
                
                union all
                
                (select      
                    e.name
                from employers e
                join vacancies v on v.employer_id = e.id
                group by e.id
                order by count(v.id) desc
                limit 1 offset 4)
'''

In [106]:
top_empl = pd.read_sql_query(query_5_1,connection)
top_empl

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name
0,Яндекс
1,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [107]:
query_5_2 = f'''select      
                    a.name "Регион",
                    count(distinct e.id) "Кол-во работодателей",
                    count(distinct v.id) "Количество вакансий"
                from areas a
                left join vacancies v on a.id = v.area_id
                left join employers e on e.area = a.id
                where v.id is null
                group by 1
                order by 2 desc
'''

In [108]:
region_stat = pd.read_sql_query(query_5_2, connection)
display(region_stat.head(10))

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Регион,Кол-во работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Республика Татарстан,16,0
8,Нижегородская область,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [109]:
query_5_3 = f'''select      
                    e.name "Работодатель",
                    count(distinct v.area_id) "Количество регионов"
                from vacancies v
                    join employers e on v.employer_id = e.id
                group by e.name
                order by 2 desc
'''

In [110]:
count_vac_in_region = pd.read_sql_query(query_5_3, connection)
display(count_vac_in_region.head(10))

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Работодатель,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [111]:
query_5_4 = f'''select 
                    e.name
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                where ei.employer_id is null
'''

In [112]:
null_ind = pd.read_sql_query(query_5_4, connection)
display(null_ind.shape[0])

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [113]:
query_5_5 = f'''select 
                    e.name
                from employers e
                    left join employers_industries ei on ei.employer_id = e.id
                group by 1
                having count(ei.industry_id)=4
                order by 1
                limit 1 offset 2
'''

In [114]:
third_pls_comp = pd.read_sql_query(query_5_5, connection)
third_pls_comp

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [115]:
query_5_6 = f'''select 
                    COUNT(*)
                from employers e
                left join employers_industries ei on ei.employer_id = e.id
                join industries i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
                group by i.name
'''

In [116]:
df_1 = pd.read_sql_query(query_5_6, connection)
df_1

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [117]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
list_city = page.find_all('table') # находим все таблицы на странице
list_city = list_city[1] # выделяем интересующую нас таблицу городов-миллиоников
lst_city = [] # создаем список для занесения данных
for i in list_city.find_all('td'): # перепираем все вхождения 'td' в таблице
    title = i.text  # значением переводим в текст
    lst_city.append(title) # полученные результаты заносим в список
lst_city = lst_city[1::5] # в полученном списке выделяем только наши города без других данных таблицы
lst_city[13] = 'Воронеж' # заменим Воронеж[а] на Воронеж
lst_city = tuple(lst_city)
print(lst_city)
    

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [118]:
query_5_7 = f'''select 
                    a.name "Город миллионик",
                    count(v.id) "Количество вакансий"
                from vacancies v
                    join areas a on v.area_id = a.id
                    join employers e on e.id = v.employer_id
                where e.name = 'Яндекс' and a.name in {lst_city}
                group by 1

                union all

                select 
                    'Total',
                    count(*)
                from vacancies v
                    join areas a on v.area_id = a.id
                    join employers e on e.id = v.employer_id
                where e.name = 'Яндекс' and a.name in {lst_city}
                group by 1
                order by 2 
'''

In [119]:
ya_lst = pd.read_sql_query(query_5_7, connection)
ya_lst

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Город миллионик,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

*Исходя мз выше проведенного анализа можно сделать вывод о том, что большинство вакансий от одной и тойже компании в нескольких регионах, в связи с чем, возможно что данная вакансия подразумевает под собой возможность работать удаленно. Также многие работадатели не указывают сферу деятельности. Самыми активными работодателями в IT направлении является "Яндекс", "Ростелеком", которые являются одними из групных IT компаний в России. В основном большинство компаний указывает 1  и более сферы деятельности в своей вакансии.* 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [120]:
query_6_1 = f'''select  
                    count(name)
                from vacancies
                where lower(name) like '%data%' or lower(name) like '%данн%'
'''

In [121]:
vac_data = pd.read_sql_query(query_6_1, connection)
vac_data

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [122]:
query_6_2 = f'''select  
                    count(name)
                from vacancies
                where (lower(name) like '%data scientist%' 
                    or lower(name) like '%data science%' 
                    or lower(name) like '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or lower(name) like '%machine learning%' 
                    or lower(name) like '%машинн%обучен%' )
                        and (lower(experience) like '%нет опыта%' 
                        or lower(employment) like '%стажировка%' 
                        or lower(name) like '%junior%')
'''

In [123]:
junior_data = pd.read_sql_query(query_6_2, connection)
junior_data

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [124]:
query_6_3 = f'''select  
                    count(name)
                    from vacancies
                    where (lower(name) like '%data scientist%' 
                        or lower(name) like '%data science%' 
                        or lower(name) like '%исследователь данных%' 
                        or (name like '%ML%' and name not like '%HTML%') 
                        or lower(name) like '%machine learning%' 
                        or lower(name) like '%машинн%обучен%' )
                            and (lower(key_skills) like '%sql%' 
                            or lower(key_skills) like '%postgres%')
'''

In [125]:
key_skills_1 = pd.read_sql_query(query_6_3, connection)
key_skills_1

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [126]:
query_6_4 = f'''select  
                    count(name)
                    from vacancies
                    where (lower(name) like '%data scientist%' 
                        or lower(name) like '%data science%' 
                        or lower(name) like '%исследователь данных%' 
                        or (name like '%ML%' and name not like '%HTML%') 
                        or lower(name) like '%machine learning%' 
                        or lower(name) like '%машинн%обучен%' )
                            and (lower(key_skills) like '%python%')
'''

In [127]:
key_skills_py = pd.read_sql_query(query_6_4, connection)
key_skills_py

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [128]:
query_6_5 = f'''select  
                    round(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills, CHR(9), ''))+ 1), 2)
                from vacancies
                where (lower(name) like '%data scientist%' 
                    or lower(name) like '%data science%' 
                    or lower(name) like '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or lower(name) like '%machine learning%' 
                    or lower(name) like '%машинн%обучен%' )
                        and key_skills is not null
'''

In [129]:
avg_kye_skills = pd.read_sql_query(query_6_5, connection)
avg_kye_skills

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [130]:
query_6_6 = f'''select  
                    experience,
                    avg(coalesce(((salary_from + salary_to) /2), salary_from, salary_to, 0)) salary
                from vacancies
                where (lower(name) like '%data scientist%' 
                    or lower(name) like '%data science%' 
                    or lower(name) like '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or lower(name) like '%machine learning%' 
                    or lower(name) like '%машинн%обучен%' )
                        and (salary_from is not null or salary_to is not null)
                    group by experience
'''

In [131]:
avg_salary_by_exp = pd.read_sql_query(query_6_6, connection)
avg_salary_by_exp

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,experience,salary
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


***

*Из выше проведенного анализа можно сделать вывод о том, что в IT специальностях в среднем необходимо 6 ключевых навыков для работы, множество IT специальностей под собой подразумевают знание одного из известных языков программирования. Также наглядно видимо, что не много вакансий ищут junior програмистов, но их средняя зарплата начинается гораздо выше чем средняя по иным специальностям. Исходя из проведенного анализа для специальностей DS мы видимо что каждые 3 года в среднем мы можем расчитывать на повышение зарплаты примерно в 70 тысяч рублей.*

# Общий вывод по проекту

- *Исходя из проделанного анализа, мы можем сделать вывод о том, что на основе данных у нас имеется 49197 вакансий, в которых представленно 23501 работадатель, представленнных 1362 регионах и 294 сферах деятельности. По количеству вакансий размешенных в регионе, являются самыми крупными "Москва", "Санкт-Питербург", также из анализа мы видим что не все работадатели заполняют поле зарплаты (salary_from, salary_to), в нашем случае одно из полей заполнено у 24073 вакансий. Можно оценить что средняя зарплата начинается от 71065 и достигает 110537 рублей. Если говорить о том, какой график представляют работодатели, то самым частым в наших данных является "Полный день- Полная занятость" с результатом 35367 вакансий, затем идет "Удаленная работа - Полная занятость" с результатом 7802, из чего можно сделать вывод, что многие работадатели перешли на удаленный график работы, после короновирусных ограничений, и тенденция идет к увелечению удаленных вакансий. Тенденци о соискателях с опытом работы сохраняется, большинство работодателей предпочитают нанимать опытных сотрудником, чтобы не тратить время на их обучение.*
- *Самыми частыми работодателями представляющими свои вакансии является "Яндекс", "Ростелеком", "Тинькофф". В нашем варианте преимущественно вакансии из городов России. Также многие работодатели не указывают сферу своей деятельности (8419), что также для соискателей может быть не очень удобно, по причине того что они могут не понять чем конкретно занимается данная компания. Но некоторые из компаний указывают несколько сфер деятельности, так как данные компании развиваются в разных сферах, чтобы охватить большое количество сфер и работать комплексно.*
- *Говоря об анализе вакансий в сфере данных, можно сказать, что таких вакансий не так уж и много всего 1771 вакансия, из них всего 51 вакансия подходит начинающим аналитикам данных. Можно понять что одним из ключевых навыков является SQL или postgers, а также python  так как эти навыки необходимы в 201 и 351 вакансии соответственно. Сотрудник работающий с данными должен обладать большим количеством ключевых навыков, для проведения качественной работы, из анализа мы видим, что в среднем это 6-7 ключевых навыков необходимых для данных соискателей. Говоря о зарплатах можно сделать вывод, что в среднем работодатели готовы платить от 74643 рублей до 243115 рублей работником в зависимости от их опыта работы.*

In [138]:
n = 15 # Зададим переменную для вывода топ n
# Напишем запрос удовльтворяющий условию
query_top_n = f'''select
                    v.name "Вакансия",
                    count(v.id) "Количество вакансий",
                    round(avg(coalesce(((v.salary_from + v.salary_to) /2), v.salary_from, v.salary_to, 0)), 1) "Средняя зарплата"
                from vacancies v
                where v.salary_from is not null and v.salary_to is not null
                group by v.name
                order by 2 desc
                limit {n}
'''

In [139]:
top_15 = pd.read_sql_query(query_top_n, connection)
top_15

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Вакансия,Количество вакансий,Средняя зарплата
0,Системный администратор,540,49448.5
1,Программист 1С,314,128984.8
2,Специалист технической поддержки,168,44108.5
3,Специалист службы поддержки Рекламной сети Янд...,139,42500.0
4,Инженер-программист,108,54632.7
5,Программист 1C,100,99509.6
6,Аналитик,82,60395.2
7,Программист,82,63844.9
8,Бизнес-аналитик,68,90975.0
9,Инженер технической поддержки,50,48867.9


*Исходя из полученного запроса мы можем сделать вывод о топ 15 вакансий по количеству размещенных на сайте предложений, с их средней запрплатой(исключая те у которых не заполнены поля зарплаты). Из представленной таблицы мы видим, что самой популярной является "Системный администратор" с результатом 540 вакансий, но и одной из самых маленьких средних зарлат.*

In [134]:
x = 'Новосибирск'
m = 'Кемерово'
query_most_paid = f'''(select
                            v.name "Вакансия",
                            count(v.id) "Количество вакансий",
                            round(avg(coalesce(((v.salary_from + v.salary_to) /2), v.salary_from, v.salary_to, 0)), 1) "Средняя зарплата",
                            a.name city
                        from vacancies v
                        join areas a on a.id = v.area_id
                        where (v.salary_from is not null and v.salary_to is not null) and a.name = '{x}'
                        group by v.name, a.name
                        order by 3 desc
                        limit 1)

                        union all 

                        (select
                                v.name "Вакансия",
                                count(v.id) "Количество вакансий",
                                round(avg(coalesce(((v.salary_from + v.salary_to) /2), v.salary_from, v.salary_to, 0)), 1) "Средняя зарплата",
                                a.name city
                        from vacancies v
                        join areas a on a.id = v.area_id
                        where (v.salary_from is not null and v.salary_to is not null) and a.name = '{m}'
                        group by v.name, a.name
                        order by 3 desc
                        limit 1)
'''

In [135]:
most_paid = pd.read_sql_query(query_most_paid, connection)
most_paid

c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,Вакансия,Количество вакансий,Средняя зарплата,city
0,Lead of Mobile,1,460000.0,Новосибирск
1,Middle PHP разработчик (middle backend php dev...,1,206500.0,Кемерово


*С целью получения данных по топ оплачивыаемых работ в конкретном городе можно использовать данный запрос, так же его можно сравнить зарплаты в разных регионов. Так из представленного примера, возможно увидеть, что ТОП-1 в "Кемерово" и "Новосибирске" являются лид в мобильной разработке и мидл php разработки, и соответствующие им зарплаты.* 

In [140]:
connection.close()